Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [1]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
import copy
from copy import deepcopy

## The *Nim* and *Nimply* classes

In [2]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [3]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [4]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [5]:
# 2.1 agent in slides
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [6]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}

In [7]:
import numpy as np

# nim_sum: function often used in the mathematical analysis of the game to find winning strategies.
# https://www.archimedes-lab.org/How_to_Solve/Win_at_Nim.html#:~:text=To%20win%20at%20Nim%2Dgame,error%20in%20order%20to%20win.
def nim_sum(state: Nim) -> int:
    # tmp: for each row, transform the number of object in that row to its binary form (width 32 bit), binary as a tuple of bits
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2) # convert the string of bit in its integer form


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply

## Evolution strategies

In [10]:
# poss_state = state + a move

width = 8 # using sigma=1 -> normal range approx between -4 and 4 ...
sigma_row = 1/width # ... now in range approx of [-1, 1], using /width we can scale the gaussian after 
sigma_num_objects = 1/width

def fitness(poss_state: Nim) -> int:
    return -nim_sum(poss_state)

def tweak(state):
    new_state = copy(state)
    new_state[index] = not new_state[index]
    return new_state

def tweak2(state, row, num_objects):
    n_rows = len(state.rows)    
    row_es = -1
    num_objects_es = -1
    
    while row_es not in range(len(state.rows)): # generate until row_es is a valid one
        row_es = round(row + np.random.normal(loc=0, scale=sigma_row*n_rows)) # n_rows is like a scaling factor for sigma

    while (num_objects_es<0) or (num_objects_es>state._k): # generate until num_objects_es is a valid one
        num_objects_es = round(num_objects + np.random.normal(loc=0, scale=sigma_row*state.rows[row_es]))
    
    return row_es, num_objects_es

### (1, 1) - ES

In [11]:
# random moves + a (1,1) variation

def es_1_1(state: Nim) -> Nimply:
    # I tweak the state: if better fitness, return move, otherwise random move

    '''Random move'''
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    poss_state = Nimply(row, num_objects)

    '''Gaussian evolution strategy''' ''' -> DO TWEAK ON STATE''' # TO DEBUG
    # generate new row with an es
    row_es, num_objects_es = tweak2(state, row, num_objects)
    poss_state_es = Nimply(row_es, num_objects_es)

    # check fitness
    if fitness(poss_state_es) > fitness(poss_state):
        return poss_state_es
    else:
        return poss_state

## Oversimplified match

In [81]:
logging.getLogger().setLevel(logging.INFO)

strategy = (pure_random, optimal)

nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    #logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    #logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")

INFO:root:init : <1 3 5 7 9 11 13 15 17 19>


INFO:root:status: Player 1 won!
